In [1]:
import warnings
warnings.filterwarnings('ignore')
#imports
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import pickle
import operator
from wordcloud import WordCloud
from PIL import Image
import string
import re
from textblob import TextBlob
import math
import seaborn as sns

In [61]:
song_links[0]

'https://www.lyricsfreak.com/p/pink+floyd/a+great+day+for+freedom_20108633.html'

In [88]:
def scrape_lyrics(links,index):
    
    # links = song_links
    url=links[index]
    url=url.replace(' ','+')
    html = urlopen(url) 
    soup = BeautifulSoup(html, 'html.parser')
    
    lyric=soup.find_all('div',class_="lyrictxt js-lyrics js-share-text-content")[0]
    
    song_lyrics=[]

    count=0

    for i in list(lyric):
        try:
            song_lyrics.append(i.replace('  ','')[1:])
        except:
             count=count+1
    
    clear=[lyr for lyr in song_lyrics if lyr!='']
    
    return clear

In [67]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [188]:
def split_text(text, n):
    length = len(text)
    size = math.floor(length / n)
    start = np.arange(0, length, size)
    split_list = []
    for piece in range(n):
        split_list.append(text[start[piece]:start[piece]+size])
    return split_list

In [226]:
def sentiment_over_song(df,polar_subject,song_title=None):
    
    chunk_sent=[]
    style=''
    
    rl_nm=[w.lower() for w in song_title.split()]
    rl_nm=' '.join(rl_nm)
    index=df[df.song==rl_nm].index[0]

    if polar_subject == 'p':
        style='polarity'
        for chunk in df.chunks[index]:
            chunk_sent.append(TextBlob(' '.join(chunk)).sentiment.polarity)
    else:
        style='subjectivity'
        for chunk in df.chunks[index]:
            chunk_sent.append(TextBlob(' '.join(chunk)).sentiment.subjectivity)

    name=[word.title() for word in df.song[index].split()]
    name=' '.join(name)
            
    y = chunk_sent
            
    x = range(0,len(df.chunks[index]))
    
    
    
    print(f'lyrics per chunk: {len(df.chunks[index])}')
    print('-'*20)
    
    with sns.axes_style({'axes.facecolor':'k','axes.edgecolor':'r'}):
        plt.figure(figsize=(15,8))
        sns.lineplot(x,y,color='gold')
        plt.title(f'Song: {name}')
        plt.xlabel('<-- time of song -->')
        plt.ylabel(f'<-- {style} -->')
        plt.rcParams.update({'font.size': 25})
        plt.tight_layout()
        plt.show()

In [235]:
def generate_sentiment(band,song_name,sent_style,lyric_breaks):    
    
    band2 = [b.lower for b in band.split()]
    band2=' '.join(band)
    
    band=band.replace(' ','+')
    band=[b.lower() for b in band.split()]
    band=''.join(band)

    url = f'https://www.lyricsfreak.com/b/{band}/'
    html = urlopen(url) 
    soup = BeautifulSoup(html, 'html.parser')

    songs=soup.find_all('div',class_="lf-list__cell lf-list__title lf-list__cell--full")

    replace_text="https://www.lyricsfreak.com"

    song_links = []
    
    ns=[]
    for song in songs:
        song=song.find_all('a')[0]['title'].split()[0:-1]
        song=' '.join(word.lower() for word in song)
        song=song.replace(' ','+')
        ns.append(song)

    replace_text="https://www.lyricsfreak.com"

    song_links = []

    for song in ns:
        sl=replace_text+song
        song_links.append(sl)
    
    song_links = []    
    
    for song in songs:
        try:
            sl=song.find_all('a')[0]['href']
            sl=replace_text+sl
            song_links.append(sl)
        except:
            p=12

    song_names_lst=[]
    song_names_dict={}

    i=0

    for song in song_links[0:-6]:
        song_names_lst.append(song.replace(f'https://www.lyricsfreak.com/b/{band}/','').split('_')[0].replace('+',' '))
        song_names_dict[i]=song.replace(f'https://www.lyricsfreak.com/b/{band}/','').split('_')[0].replace('+',' ')
        i+=1

    df=pd.DataFrame()

    song_links=song_links[0:-6]

    df['index']=song_names_dict.keys()

    df['song']=song_names_dict.values()
    
    for i in range(len(df)):
        if band2 not in df.song[i]:
            df.song[i]='nosong'
    df=df[df.song!='nosong']
    
    df['lyrics']=df['index'].apply(lambda x: scrape_lyrics(x,'c',song_links))

    df.drop('index',axis=1,inplace=True)

    eng_stop=stopwords.words('english')

    df.reset_index(inplace=True)

    df.drop('index',axis=1,inplace=True)

    df['lyrics_mush']=df.lyrics.apply(lambda x: ' '.join(x))

    df['clean_lyrics']=df.lyrics_mush.apply(lambda x: clean_text(x))

    df.drop('lyrics_mush',axis=1,inplace=True)

    df['no_stop_lyrics']=None
    for i in range(len(df)):
        df['no_stop_lyrics'][i]=[lyric for lyric in df.clean_lyrics[i].split() if lyric not in eng_stop]

    for i in range(len(df)):
        df.clean_lyrics[i]=' '.join(df['no_stop_lyrics'][i])

    df.drop('lyrics',axis=1,inplace=True)

    pol = lambda x: TextBlob(x).sentiment.polarity
    sub = lambda x: TextBlob(x).sentiment.subjectivity

    df['polarity'] = df['clean_lyrics'].apply(pol)
    df['subjectivity'] = df['clean_lyrics'].apply(sub)
    
    #df.drop('level_0',axis=1,inplace=True)

    df['chunks']=df.no_stop_lyrics.apply(lambda x: split_text(x,lyric_breaks))
    
    return sentiment_over_song(df,sent_style,song_name)

In [236]:
def interactive_song_monitor():
    pass

### STOPPED HERE

In [246]:
url = f'https://www.lyricsfreak.com/b/pink+floyd/'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

songs=soup.find_all('div',class_="lf-list__cell lf-list__title lf-list__cell--full")

replace_text="https://www.lyricsfreak.com"

song_links = []

ns=[]
for song in songs:
    song=song.find_all('a')[0]['title'].split()[0:-1]
    song=' '.join(word.lower() for word in song)
    song=song.replace(' ','+')
    ns.append(song)

In [ ]:
replace_text="https://www.lyricsfreak.com"

song_links = []

for song in ns:
    sl=replace_text+song
    song_links.append(sl)

In [ ]:
song_links = []    

for song in songs:
    try:
        sl=song.find_all('a')[0]['href']
        sl=replace_text+sl
        song_links.append(sl)
    except:
        p=12

song_links=song_links[0:-6]

In [ ]:
song_names_lst=[]
song_names_dict={}

i=0

for song in song_links:
    song_names_lst.append(song.replace('https://www.lyricsfreak.com/b/pink+floyd/','').split('_')[0].replace('+',' '))
    song_names_dict[i]=song.replace('https://www.lyricsfreak.com/b/pink+floyd/','').split('_')[0].replace('+',' ')
    i+=1

In [247]:
df=pd.DataFrame()

df['index']=song_names_dict.keys()

df['song']=song_names_dict.values()

In [199]:
## remove lyrics that are not from the band
for i in range(len(df)):
    if 'pink floyd' not in df.song[i]:
        df.song[i]='nosong'

In [200]:
df=df[df.song!='nosong']

In [201]:
df['lyrics']=df['index'].apply(lambda x: scrape_lyrics(song_links,x))

In [202]:
eng_stop=stopwords.words('english')

df.reset_index(inplace=True)

df.drop('index',axis=1,inplace=True)

df['lyrics_mush']=df.lyrics.apply(lambda x: ' '.join(x))

df['clean_lyrics']=df.lyrics_mush.apply(lambda x: clean_text(x))

df.drop('lyrics_mush',axis=1,inplace=True)

df['no_stop_lyrics']=None
for i in range(len(df)):
    df['no_stop_lyrics'][i]=[lyric for lyric in df.clean_lyrics[i].split() if lyric not in eng_stop]

for i in range(len(df)):
    df.clean_lyrics[i]=' '.join(df['no_stop_lyrics'][i])

df.drop('lyrics',axis=1,inplace=True)

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df['polarity'] = df['clean_lyrics'].apply(pol)
df['subjectivity'] = df['clean_lyrics'].apply(sub)

df.drop('level_0',axis=1,inplace=True)

df['chunks']=df.no_stop_lyrics.apply(lambda x: split_text(x,1))

In [203]:
df

,song,clean_lyrics,no_stop_lyrics,polarity,subjectivity,chunks
0,https://www.lyricsfreak.com/p/pink floyd/a gre...,day wall came threw locks onto ground glasses ...,"[day, wall, came, threw, locks, onto, ground, ...",0.072857,0.417619,"[[day, wall, came, threw, locks, onto, ground,..."
1,https://www.lyricsfreak.com/p/pink floyd/a new...,always always looked behind eyes feels like li...,"[always, always, looked, behind, eyes, feels, ...",-0.333333,0.683333,"[[always, always, looked, behind, eyes, feels,..."
2,https://www.lyricsfreak.com/p/pink floyd/a new...,always always looked behind eyes feels like li...,"[always, always, looked, behind, eyes, feels, ...",-0.333333,0.683333,"[[always, always, looked, behind, eyes, feels,..."
3,https://www.lyricsfreak.com/p/pink floyd/a new...,always always look behind eyes lifetime lifeti...,"[always, always, look, behind, eyes, lifetime,...",-0.400000,0.700000,"[[always, always, look, behind, eyes, lifetime..."
4,https://www.lyricsfreak.com/p/pink floyd/a pil...,cloud eider draws around softening sound sleep...,"[cloud, eider, draws, around, softening, sound...",0.150000,0.464103,"[[cloud, eider, draws, around, softening, soun..."
...,...,...,...,...,...,...
162,https://www.lyricsfreak.com/p/pink floyd/wots ...,heaven said promised land looks allright stand...,"[heaven, said, promised, land, looks, allright...",0.081122,0.444218,"[[heaven, said, promised, land, looks, allrigh..."
163,https://www.lyricsfreak.com/p/pink floyd/yet a...,one sound one single sound one kiss one single...,"[one, sound, one, single, sound, one, kiss, on...",-0.007811,0.360276,"[[one, sound, one, single, sound, one, kiss, o..."
164,https://www.lyricsfreak.com/p/pink floyd/you g...,gotta crazy gotta mean gotta keep shoes car cl...,"[gotta, crazy, gotta, mean, gotta, keep, shoes...",0.047867,0.545959,"[[gotta, crazy, gotta, mean, gotta, keep, shoe..."
165,https://www.lyricsfreak.com/p/pink floyd/young...,new boy stranger town good times whos gonna sh...,"[new, boy, stranger, town, good, times, whos, ...",-0.216364,0.695455,"[[new, boy, stranger, town, good, times, whos,..."


In [ ]:
' '.join(word.lower() for word in songs[0].find_all('a')[0]['title'].split()[0:-1]).replace(' ','+')

In [110]:
ns=[]
for song in songs:
    song=song.find_all('a')[0]['title'].split()[0:-1]
    song=' '.join(word.lower() for word in song)
    song=song.replace(' ','+')
    ns.append(song)
    
replace_text="https://www.lyricsfreak.com"

song_links = []

for song in ns:
    sl=replace_text+song
    song_links.append(sl)